<a href="https://colab.research.google.com/github/Birkbeck/bsc-computer-science-project-2021_22-mohammadreza490/blob/main/project_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bass Extractor - Project Notebook

## Installing and Importing Dependencies

In [1]:
# Relative path to src folder here
PATH_TO_SRC_FOLDER = r".\src"

In [2]:
# Allows auto reloading of modules
%load_ext autoreload
%autoreload 2

In [ ]:
# Install dependencies
!pip install musdb
!pip install numpy
!pip install pydub
!pip install "tensorflow<2.11" # GPU can only be used for versions 2.10 and earlier
!pip install ipdb
!pip install museval
!pip install librosa
!pip install matplotlib

In [3]:
# Import modules
import os
import sys

import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import IPython.display as ipd
import musdb
import pydub
import scipy
import soundfile as sf
import museval
import random
import math

from scipy.io.wavfile import write
from glob import glob

In [4]:
# Add path to src folder to the path variable 
if PATH_TO_SRC_FOLDER not in sys.path:
  sys.path.append(PATH_TO_SRC_FOLDER) # https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab

In [10]:
# Import user-defined modules
from wav_handler import WavHandler
from dataset_handler import DatasetHandler
from plotter import Plotter
from model_handler import ModelHandler
from config import INPUT_WIDTH, INPUT_HEIGHT, INPUT_CHANNELS, OUTPUT_CHANNELS, PATH_TO_MODEL

In [6]:
# Check if TensorFlow is properly installed and if GPU is detected
print(tf.reduce_sum(tf.random.normal([1000, 1000])))
print(tf.config.list_physical_devices('GPU'))

tf.Tensor(-1982.6267, shape=(), dtype=float32)
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Building the Model

In [11]:
plotter = Plotter()
model_handler = ModelHandler()
dataset_handler = DatasetHandler()

In [12]:
unet = model_handler.buildModel()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1040, 176,   0           []                               
                                1)]                                                               
                                                                                                  
 conv2d_19 (Conv2D)             (None, 1040, 176, 3  320         ['input_2[0][0]']                
                                2)                                                                
                                                                                                  
 batch_normalization_18 (BatchN  (None, 1040, 176, 3  128        ['conv2d_19[0][0]']              
 ormalization)                  2)                                                          

## Preparing the Dataset

In [ ]:
# dataset_handler.writeTrainInputSpec()
# dataset_handler.writeTrainOutputSpec()
# dataset_handler.createBassless()

## Training the Model

In [13]:
callbacks = model_handler.createCallbacks()
train_dataset =  model_handler.buildDataFromGenerator()
results = model_handler.modelFit(unet, callbacks, train_dataset)

Epoch 1/40


c:\Users\Rohan\miniconda3\envs\tf\lib\site-packages\tensorflow\python\ops\script_ops.py:230: ComplexWarning: Casting complex values to real discards the imaginary part
  result = np.asarray(value, dtype=dtype, order="C")


   1436/Unknown - 684s 469ms/step - loss: -0.7096 - accuracy: 0.6437
Epoch 1: saving model to ./model\bass_separator_40e_8b.h5
1436/1436 [==============================] - 685s 469ms/step - loss: -0.7096 - accuracy: 0.6437
Epoch 2/40
1436/1436 [==============================] - ETA: 0s - loss: -0.8351 - accuracy: 0.6886
Epoch 2: saving model to ./model\bass_separator_40e_8b.h5
1436/1436 [==============================] - 629s 438ms/step - loss: -0.8351 - accuracy: 0.6886
Epoch 3/40
1436/1436 [==============================] - ETA: 0s - loss: -0.8413 - accuracy: 0.7003
Epoch 3: saving model to ./model\bass_separator_40e_8b.h5
1436/1436 [==============================] - 574s 400ms/step - loss: -0.8413 - accuracy: 0.7003
Epoch 4/40
1436/1436 [==============================] - ETA: 0s - loss: -0.8783 - accuracy: 0.7177
Epoch 4: saving model to ./model\bass_separator_40e_8b.h5
1436/1436 [==============================] - 546s 380ms/step - loss: -0.8783 - accuracy: 0.7177
Epoch 5/40
1436/14

## Validating the Model

In [18]:
song_path = "Music/9mm Parabellum Bullet - Heart Ni Hi Wo Tsukete (Dawning Version) (44100Hz).wav"
model_name = "bass_separator_40e_8b.h5"

bass, bassless = model_handler.predictSong(song_path, model_name)

list = song_path.split("/")
song_name = list[-1][:-4]

bass_istft, bassless_istft = model_handler.getOutputWavs(song_path, bass, bassless)

model_handler.saveOutputs(bass_istft, bassless_istft, song_name)

Sampling rate:  44100
Number of segments:  120
Spectrograms shape:  (120, 1025, 173)
Padded spectrograms shape:  (120, 1040, 176)

15/15 [==============================] - 2s 92ms/step
Output shape:  (120, 1040, 176, 2)
Output shape (swapped axes):  (2, 120, 1040, 176)
Bass output shape:  (120, 1040, 176)
Bassless output shape:  (120, 1040, 176)

